### 推理
若有需要推理的模型，则继续运行下面的代码

In [1]:
# 若是一路运行下来的，这个框不用运行
import pandas as pd
from tqdm import tqdm
import json
datapath = '/data/lj/llm/dataset/iter4/iter4-clean.csv'
infered_data = pd.read_csv(datapath)
infered_data.head(1)

,q,que,kb_ids,new_refer,actual_send_answer,qwl02-cleaned-base,qwl02-cleaned-base-gpt35,qwl02-cleaned-chat,qwl02-cleaned-chat-gpt35,qwl02-cleaneddoc3,...,yil02-cleandoc-base,yil02-cleandoc-Chat-34B,yil02-cleandoc-Chat-34B_gpt35,qwl02,qwl02_gpt35,qwl02-cleaneddoc8-32b-base,qwl02-cleaneddoc8-32b-base_gpt35,qwl02-cleaneddoc8-32b-base-new,qwl02-cleaneddoc13-32b-base,qwl02-cleaneddoc18-32b-base
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","22, 23","['问题：""一包肠衣灌几斤肉啊"",答案：""亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌...","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...",亲，猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,"亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",...,亲，一包肠衣口径10斤左右，口径3.8-4.0左右，灌大约8-10斤肉。根据个人手法，会有误...,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左右，...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，...,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。"


In [2]:
def process_data(df):
    """
    csv 多列 -> json 一条一条数据的格式
    数据梳理为可以直接让模型推理的格式
    """
    INSTRUCTION = ("你是一名电商人工客服，请用礼貌、专业的态度回答用户的问题，请务必使用中文回答。必要时请参考对话记录。\n"
               "以下是供你参考的已知信息：\n{KNOWLEDGE}\n"
               "上述的已知信息可能与用户的问题无关，你需要根据用户的具体需求选择性的参考。\n"
               "另外你要避免向客户做出承诺，如优惠、物流时间等\n\n"
               "你要回复的问题是：")
    querys = []
    answers = []
    historys = []
    knowledge = []
    
    df_shop = df[:400]  # 200   200
    df_doc = df[400:] # 100 pub-doc

    # 店铺数据测试集
    for index, row in tqdm(df_shop.iterrows()):
        if len(eval(row['que'])) > 0:
            query = eval(row['que'])[-1]['content'].strip()
            querys.append(eval(row['que'])[-1])
        else:
        # 处理列表为空的情况，例如设置一个默认值或者跳过这一行
            query = None
        history = eval(row['que'])[:-1]
        if history:
            temp = [history[0].copy()]
            for i, record in enumerate(history[1:]):
                if record["role"] == temp[-1]["role"]:
                    temp[-1]["content"] += '\n' + record["content"]
                else:
                    temp.append(record.copy())
        else:
            temp = ''
        historys.append(temp)
        answers.append(str(row['actual_send_answer']))
        # print(row['refer_content'])
        kg = json.loads(row['new_refer'].replace('\"', '\\\"').replace('\'', '\"'))
        kg = kg[:6]
        knowledge.append("\n".join([f"{i+1}. {qa}" for i, qa in enumerate(kg)]))
    assert len(historys) == len(querys) == len(answers)
    eval_data = []
    for history,query,answer,kg in tqdm(zip(historys,querys,answers,knowledge)):
        eval_data.append({
            "instruction":INSTRUCTION.format_map({"KNOWLEDGE": kg}),
            "output":answer,
            "history":[[x['role']+': '+ x['content'] for x in history]],
            "input":query['content']
        })

     # 通用docqa测试集
    for index, row in tqdm(df_doc.iterrows()):
        eval_data.append({
            "instruction":row['new_refer'],
            "output":row['actual_send_answer'],
            "history":row['que'],
            "input":row['q']
        })
        
    return eval_data
# 列数据优化为可直接进行推理格式
infer_data = process_data(infered_data)
infer_data[0]

400it [00:00, 5105.09it/s]
400it [00:00, 286300.61it/s]
100it [00:00, 20909.84it/s]


{'instruction': '你是一名电商人工客服，请用礼貌、专业的态度回答用户的问题，请务必使用中文回答。必要时请参考对话记录。\n以下是供你参考的已知信息：\n1. 问题："一包肠衣灌几斤肉啊",答案："亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。"\n2. 问题："一袋肠衣能做几斤肉",答案："猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。"\n3. 问题："100g肠衣能灌多少肉",答案："亲 不清楚哦"\n4. 问题："一包猪肠子适合几斤肉？",答案："猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。"\n5. 问题："这一包肠衣可以灌多少斤肉？",答案："猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。"\n上述的已知信息可能与用户的问题无关，你需要根据用户的具体需求选择性的参考。\n另外你要避免向客户做出承诺，如优惠、物流时间等\n\n你要回复的问题是：',
 'output': '猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。',
 'history': [['assistant: 很高兴为您服务,请问有什么可以帮您？']],
 'input': '一包肠依做多少肉'}

In [4]:

from infer import infer
import torch

model_path = '/data/lj/models/Qwen1.5-32B/qwl02-cleaneddoc18-32b-base-cp12000'

model_name = model_path.split('/')[-1]
# model_name ='yil02-cleandoc-Chat'
result = infer(model_path,infer_data,None)
assert len(infer_data) ==  len(result)
infered_data[model_name] = result
torch.cuda.empty_cache()


save_path = datapath
infered_data.to_csv(save_path,index=False)
print('写入完毕')
infered_data.head(1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  2%|▏         | 1/50 [00:05<04:22,  5.35s/it]

query: 一包肠依做多少肉 ai: 亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。 lable: 猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。


  4%|▍         | 2/50 [00:09<03:29,  4.36s/it]

query: 好了不跟你计较可以多些都结了。 ai: 好的亲 lable: 你好 在的亲


  6%|▌         | 3/50 [00:11<02:51,  3.66s/it]

...

query: 台湾北部第二个直辖市于什么时候产生？ ai: 中华民国99年12月25日 lable: 中华民国99年12月25日


100%|██████████| 50/50 [04:08<00:00,  4.98s/it]

query: iphone4什么时候出的 ai: 2010年6月8日凌晨1点 lable: 2010年6月8日凌晨1点
done


写入完毕


,q,que,kb_ids,new_refer,actual_send_answer,qwl02-cleaned-base,qwl02-cleaned-base-gpt35,qwl02-cleaned-chat,qwl02-cleaned-chat-gpt35,qwl02-cleaneddoc3,...,yil02-cleandoc-Chat-34B,yil02-cleandoc-Chat-34B_gpt35,qwl02,qwl02_gpt35,qwl02-cleaneddoc8-32b-base,qwl02-cleaneddoc8-32b-base_gpt35,qwl02-cleaneddoc8-32b-base-new,qwl02-cleaneddoc13-32b-base,qwl02-cleaneddoc18-32b-base,qwl02-cleaneddoc18-32b-base-cp12000
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","22, 23","['问题：""一包肠衣灌几斤肉啊"",答案：""亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌...","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...",亲，猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,"亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",...,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左右，...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，...,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。


In [5]:

from infer import infer
import torch

model_path = '/data/lj/models/Qwen1.5-32B/qwl02-cleaneddoc18-32b-base-cp15000'

model_name = model_path.split('/')[-1]
# model_name ='yil02-cleandoc-Chat'
result = infer(model_path,infer_data,None)
assert len(infer_data) ==  len(result)
infered_data[model_name] = result
torch.cuda.empty_cache()


save_path = datapath
infered_data.to_csv(save_path,index=False)
print('写入完毕')
infered_data.head(1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  2%|▏         | 1/50 [00:05<04:22,  5.35s/it]

query: 一包肠依做多少肉 ai: 亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。 lable: 猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。


  4%|▍         | 2/50 [00:08<03:16,  4.10s/it]

query: 好了不跟你计较可以多些都结了。 ai: 亲，确认收货以后联系我们帮您处理哈 lable: 你好 在的亲


  6%|▌         | 3/50 [00:11<02:44,  3.50s/it]

...

 98%|█████████▊| 49/50 [04:15<00:05,  5.28s/it]

query: 台湾北部第二个直辖市于什么时候产生？ ai: 中华民国99年12月25日 lable: 中华民国99年12月25日


100%|██████████| 50/50 [04:19<00:00,  5.20s/it]

query: iphone4什么时候出的 ai: 2010年6月8日凌晨1点 lable: 2010年6月8日凌晨1点
done
写入完毕


,q,que,kb_ids,new_refer,actual_send_answer,qwl02-cleaned-base,qwl02-cleaned-base-gpt35,qwl02-cleaned-chat,qwl02-cleaned-chat-gpt35,qwl02-cleaneddoc3,...,yil02-cleandoc-Chat-34B_gpt35,qwl02,qwl02_gpt35,qwl02-cleaneddoc8-32b-base,qwl02-cleaneddoc8-32b-base_gpt35,qwl02-cleaneddoc8-32b-base-new,qwl02-cleaneddoc13-32b-base,qwl02-cleaneddoc18-32b-base,qwl02-cleaneddoc18-32b-base-cp12000,qwl02-cleaneddoc18-32b-base-cp15000
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","22, 23","['问题：""一包肠衣灌几斤肉啊"",答案：""亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌...","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...",亲，猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,"亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，...,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。"


In [6]:

from infer import infer
import torch

model_path = '/data/lj/models/Qwen1.5-32B/qwl02-cleaneddoc18-32b-base-cp18000'

model_name = model_path.split('/')[-1]
# model_name ='yil02-cleandoc-Chat'
result = infer(model_path,infer_data,None)
assert len(infer_data) ==  len(result)
infered_data[model_name] = result
torch.cuda.empty_cache()


save_path = datapath
infered_data.to_csv(save_path,index=False)
print('写入完毕')
infered_data.head(1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  2%|▏         | 1/50 [00:06<05:21,  6.55s/it]

query: 一包肠依做多少肉 ai: 亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。 lable: 猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,口径3.8-4.0左右,灌大约8-10斤肉。


  4%|▍         | 2/50 [00:10<03:47,  4.75s/it]

query: 好了不跟你计较可以多些都结了。 ai: 亲，确认收货以后联系我们帮您处理哈 lable: 你好 在的亲


  6%|▌         | 3/50 [00:13<03:05,  3.95s/it]

...

 98%|█████████▊| 49/50 [06:58<00:11, 11.05s/it]

query: 台湾北部第二个直辖市于什么时候产生？ ai: 中华民国99年12月25日 lable: 中华民国99年12月25日


100%|██████████| 50/50 [07:05<00:00,  8.52s/it]

query: iphone4什么时候出的 ai: 2010年6月8日凌晨1点 lable: 2010年6月8日凌晨1点
done
写入完毕


,q,que,kb_ids,new_refer,actual_send_answer,qwl02-cleaned-base,qwl02-cleaned-base-gpt35,qwl02-cleaned-chat,qwl02-cleaned-chat-gpt35,qwl02-cleaneddoc3,...,qwl02,qwl02_gpt35,qwl02-cleaneddoc8-32b-base,qwl02-cleaneddoc8-32b-base_gpt35,qwl02-cleaneddoc8-32b-base-new,qwl02-cleaneddoc13-32b-base,qwl02-cleaneddoc18-32b-base,qwl02-cleaneddoc18-32b-base-cp12000,qwl02-cleaneddoc18-32b-base-cp15000,qwl02-cleaneddoc18-32b-base-cp18000
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","22, 23","['问题：""一包肠衣灌几斤肉啊"",答案：""亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌...","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...",亲，猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉；10斤一包8-10米左...,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,"亲,5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",...,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,1,猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，...,亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。",亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。,"亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。","亲，5斤一包肠衣可以灌5斤左右肉,10斤装的可以灌8-10斤肉,根据个人手法会有所不同哦。"
